# Linear Regression

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV



df = pd.read_csv("synthetic_health_data.csv")

print(df.shape)





(6005, 8)


In [2]:

target_col = "Health_Score"



y = df[target_col]


X = df.drop(columns=[target_col])


X = X.select_dtypes(include=[np.number])

print("Features shape:", X.shape)
print("Target shape:", y.shape)


Features shape: (6005, 0)
Target shape: (6005,)


In [4]:

df = pd.read_csv("synthetic_health_data.csv")


binary_cols = ["Smoking_Status"]
target_col  = "Health_Score"


cont_cols = ["Age", "BMI", "Diet_Quality", "Sleep_Hours", "Alcohol_Consumption"]

for col in cont_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce")


df["Exercise_Frequency"] = pd.to_numeric(df["Exercise_Frequency"], errors="coerce")
df["Smoking_Status"]     = pd.to_numeric(df["Smoking_Status"], errors="coerce")


df = df.dropna(subset=cont_cols + ["Exercise_Frequency", "Smoking_Status"])


df = df[df["Alcohol_Consumption"] >= 0]


for col in cont_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    low = Q1 - 1.5 * IQR
    high = Q3 + 1.5 * IQR
    df = df[(df[col] >= low) & (df[col] <= high)]


df["Diet_Exercise"]  = df["Diet_Quality"] * df["Exercise_Frequency"]
df["Sleep_Exercise"] = df["Sleep_Hours"] * df["Exercise_Frequency"]
df["BMI_sq"]         = df["BMI"] ** 2
df["Alcohol_sq"]     = df["Alcohol_Consumption"] ** 2


for col in ["BMI_Diet_Quality", "Diet_Sleep", "Alc_Execercise",
            "age_sleep", "Exercise_Sleep", "Diet_Quality_BMI"]:
    if col in df.columns:
        df = df.drop(columns=[col])


df["Smoking_Status"] = df["Smoking_Status"].astype(int)
df["Exercise_Frequency"] = df["Exercise_Frequency"].astype(int)

df = df.drop_duplicates()
X = df.drop(columns=[target_col])
y = df[target_col]

numeric_cols = [c for c in df.columns if c not in binary_cols + [target_col]]


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


preprocess = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_cols),
        ("bin", "passthrough", binary_cols),
    ]
)